# Exploratory Data analysis Trajectory


#### 1. import libaries

In [3]:
# relevant libs
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import datetime


#### 2. Import Data

In [4]:
# read in the data
traj_gdf = gpd.read_file('../scratch/traj.geojson')
picks_gdf = gpd.read_file('../scratch/picks.geojson')

#### 3. Data Wrangling

In [5]:
def create_Trajectory_object(traj_gdf):

    trajectory = mpd.Trajectory(traj_gdf, traj_id='traj_id', t='t')
    trajectory.add_acceleration()
    trajectory.add_speed()
    trajectory.add_timedelta()

    return trajectory

trajectory = create_Trajectory_object(traj_gdf)

In [6]:
def change_date_time(gdf, col_name, time_change):
    picks_gdf[col_name] = picks_gdf[col_name] + pd.to_timedelta(time_change)
    return gdf

picks_gdf = change_date_time(picks_gdf, 'date_taken', '2 hours')

#### 4. Explore data

In [7]:
print('Trajectory start time: ', traj_gdf['t'].min())
print('Trajectory end time: ', traj_gdf['t'].max())
print('Picks start time: ', picks_gdf['date_taken'].min())
print('Picks end time: ', picks_gdf['date_taken'].max())

Trajectory start time:  2023-08-19 10:08:43
Trajectory end time:  2023-08-19 16:22:21
Picks start time:  2023-08-19 14:12:31
Picks end time:  2023-08-19 20:19:32


In [8]:
traj_gdf.head()

,t,track_id,traj_id,geometry
0,2023-08-19 10:08:43,1,1,POINT (4.73950 52.64884)
1,2023-08-19 10:08:44,1,1,POINT (4.73953 52.64880)
2,2023-08-19 10:08:45,1,1,POINT (4.73953 52.64879)
3,2023-08-19 10:08:46,1,1,POINT (4.73954 52.64880)
4,2023-08-19 10:08:47,1,1,POINT (4.73955 52.64881)


In [9]:
picks_gdf.head()

,id,verification,phone,date_taken,date_uploaded,lat,lon,picked up,address,total_litter,...,elec_small,elec_large,batteries,balloons.1,life_buoy,other,custom_tag_1,custom_tag_2,custom_tag_3,geometry
0,452620,2,iPhone 6,2023-08-19 14:41:09,2023-08-20 10:55:39,52.632703,4.751589,Yes,"15A, Kanaalkade, Binnenstad-Oost, Alkmaar, Nor...",2.0,...,None,None,None,None,None,None,brand:desperados,certificate-76,None,POINT (4.75159 52.63270)
1,452621,2,iPhone 6,2023-08-19 14:49:32,2023-08-20 10:56:05,52.629392,4.751481,Yes,"87, Verdronkenoord, Binnenstad-Oost, Alkmaar, ...",2.0,...,None,None,None,None,None,None,brand:amigo,certificate-76,None,POINT (4.75148 52.62939)
2,452622,2,iPhone 6,2023-08-19 18:13:51,2023-08-20 10:56:08,52.543922,4.720822,Yes,"Geesterweg, Klein Dorregeest, Akersloot, Uitge...",2.0,...,None,None,None,None,None,None,brand:shiraz,certificate-76,None,POINT (4.72082 52.54392)
3,452623,2,iPhone 6,2023-08-19 15:18:32,2023-08-20 10:56:12,52.618136,4.757294,Yes,"302D, Koelmalaan, Overdie, Alkmaar, North Holl...",2.0,...,None,None,None,None,None,None,certificate-76,None,None,POINT (4.75729 52.61814)
4,452624,2,iPhone 6,2023-08-19 14:39:35,2023-08-20 10:56:16,52.633186,4.750136,Yes,"21A, Kanaalkade, Centrum, Alkmaar, North Holla...",4.0,...,None,None,None,None,None,None,certificate-76,None,None,POINT (4.75014 52.63319)


#### 5. Data Visualization

In [10]:
trajectory.hvplot(title='Trajectory', line_width=7.0, tiles='OSM', frame_width=700, frame_height=500)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Path.I   :Path   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (triangle_angle)

In [11]:
def plot_traj_speed(trajectory, end_time=1):
    
    # Get the start time of the trajectory
    start_time = trajectory.get_start_time()

    # Calculate the end time for the subset (e.g., 2 hours after the start)
    end_time = start_time + datetime.timedelta(hours=1)

    # Create a subset of the trajectory
    subset = trajectory.get_segment_between(start_time, end_time)

    # Plot the subset
    return subset.hvplot(c='speed', line_width=7.0, tiles='OSM', frame_width=700, frame_height=500)


plot_traj_speed(trajectory, 1)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Path.I   :Path   [Longitude,Latitude]   (speed)
   .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)

In [12]:
detector = mpd.TrajectoryStopDetector(trajectory)
stops = detector.get_stop_segments(min_duration= datetime.timedelta(seconds=15),max_diameter=10)

In [13]:
stops.hvplot(title='Stops', line_width=7.0, tiles='OSM', frame_width=700, frame_height=500) + picks_gdf.hvplot(geo=True, title='Picks', tiles="OSM", color='red', size=10, frame_width=700, frame_height=500) + trajectory.hvplot(title='Trajectory', line_width=7.0, tiles='OSM', frame_width=700, frame_height=500)

:Layout
   .Overlay.I   :Overlay
      .WMTS.I           :WMTS   [Longitude,Latitude]
      .Path.I           :Path   [Longitude,Latitude]
      .Path.II          :Path   [Longitude,Latitude]
      .Path.III         :Path   [Longitude,Latitude]
      .Path.IV          :Path   [Longitude,Latitude]
      .Path.V           :Path   [Longitude,Latitude]
      .Path.VI          :Path   [Longitude,Latitude]
      .Path.VII         :Path   [Longitude,Latitude]
      .Path.VIII        :Path   [Longitude,Latitude]
      .Path.IX          :Path   [Longitude,Latitude]
      .Path.X           :Path   [Longitude,Latitude]
      .Path.XI          :Path   [Longitude,Latitude]
      .Path.XII         :Path   [Longitude,Latitude]
      .Path.XIII        :Path   [Longitude,Latitude]
      .Path.XIV         :Path   [Longitude,Latitude]
      .Path.XV          :Path   [Longitude,Latitude]
      .Path.XVI         :Path   [Longitude,Latitude]
      .Path.XVII        :Path   [Longitude,Latitude]
      .Path.XVIII       :Path   [Longitude,Latitude]
      .Path.XIX         :Path   [Longitude,Latitude]
      .Path.XX          :Path   [Longitude,Latitude]
      .Path.XXI         :Path   [Longitude,Latitude]
      .Path.XXII        :Path   [Longitude,Latitude]
      .Path.XXIII       :Path   [Longitude,Latitude]
      .Path.XXIV        :Path   [Longitude,Latitude]
      .Path.XXV         :Path   [Longitude,Latitude]
      .Path.XXVI        :Path   [Longitude,Latitude]
      .Path.XXVII       :Path   [Longitude,Latitude]
      .Path.XXVIII      :Path   [Longitude,Latitude]
      .Path.XXIX        :Path   [Longitude,Latitude]
      .Path.XXX         :Path   [Longitude,Latitude]
      .Path.XXXI        :Path   [Longitude,Latitude]
      .Path.XXXII       :Path   [Longitude,Latitude]
      .Path.XXXIII      :Path   [Longitude,Latitude]
      .Path.XXXIV       :Path   [Longitude,Latitude]
      .Path.XXXV        :Path   [Longitude,Latitude]
      .Path.XXXVI       :Path   [Longitude,Latitude]
      .Path.XXXVII      :Path   [Longitude,Latitude]
      .Path.XXXVIII     :Path   [Longitude,Latitude]
      .Path.XXXIX       :Path   [Longitude,Latitude]
      .Path.XL          :Path   [Longitude,Latitude]
      .Path.XLI         :Path   [Longitude,Latitude]
      .Path.XLII        :Path   [Longitude,Latitude]
      .Path.XLIII       :Path   [Longitude,Latitude]
      .Path.XLIV        :Path   [Longitude,Latitude]
      .Path.XLV         :Path   [Longitude,Latitude]
      .Path.XLVI        :Path   [Longitude,Latitude]
      .Path.XLVII       :Path   [Longitude,Latitude]
      .Path.XLVIII      :Path   [Longitude,Latitude]
      .Path.XLIX        :Path   [Longitude,Latitude]
      .Path.L           :Path   [Longitude,Latitude]
      .Path.LI          :Path   [Longitude,Latitude]
      .Path.LII         :Path   [Longitude,Latitude]
      .Path.LIII        :Path   [Longitude,Latitude]
      .Path.LIV         :Path   [Longitude,Latitude]
      .Path.LV          :Path   [Longitude,Latitude]
      .Path.LVI         :Path   [Longitude,Latitude]
      .Path.LVII        :Path   [Longitude,Latitude]
      .Path.LVIII       :Path   [Longitude,Latitude]
      .Path.LIX         :Path   [Longitude,Latitude]
      .Path.LX          :Path   [Longitude,Latitude]
      .Path.LXI         :Path   [Longitude,Latitude]
      .Path.LXII        :Path   [Longitude,Latitude]
      .Path.LXIII       :Path   [Longitude,Latitude]
      .Path.LXIV        :Path   [Longitude,Latitude]
      .Path.LXV         :Path   [Longitude,Latitude]
      .Path.LXVI        :Path   [Longitude,Latitude]
      .Path.LXVII       :Path   [Longitude,Latitude]
      .Path.LXVIII      :Path   [Longitude,Latitude]
      .Path.LXIX        :Path   [Longitude,Latitude]
      .Path.LXX         :Path   [Longitude,Latitude]
      .Path.LXXI        :Path   [Longitude,Latitude]
      .Path.LXXII       :Path   [Longitude,Latitude]
      .Path.LXXIII      :Path   [Longitude,Latitude]
      .Path.LXXIV       :Path   [Longitude,La

In [14]:
stops

TrajectoryCollection with 222 trajectories

In [15]:
time_stops = detector.get_stop_time_ranges(min_duration=datetime.timedelta(seconds=15), max_diameter=10)

In [16]:
median_pace = np.median(trajectory.get_length() / trajectory.get_duration().total_seconds())
median_pace_kmph = median_pace * 3.6
print(f"the median pace is {median_pace_kmph:.2f} km/h")
print(f"the median pace is {median_pace:.2f} m/s")




the median pace is 4.44 km/h
the median pace is 1.23 m/s


In [18]:
trajectory.to_point_gdf().head()

,track_id,traj_id,geometry,acceleration,speed,timedelta
t,,,,,,
2023-08-19 10:08:43,1,traj_id,POINT (4.73950 52.64884),0.000000,4.451457,NaT
2023-08-19 10:08:44,1,traj_id,POINT (4.73953 52.64880),0.000000,4.451457,0 days 00:00:01
2023-08-19 10:08:45,1,traj_id,POINT (4.73953 52.64879),-3.547671,0.903786,0 days 00:00:01
2023-08-19 10:08:46,1,traj_id,POINT (4.73954 52.64880),0.138177,1.041962,0 days 00:00:01
2023-08-19 10:08:47,1,traj_id,POINT (4.73955 52.64881),0.130245,1.172208,0 days 00:00:01


In [ ]:
trajectory.to_point_gdf().to_file('../scratch/trajectory_points.geojson', driver='GeoJSON')

ValueError: Invalid field type <class 'pandas._libs.tslibs.timedeltas.Timedelta'>

#### 6. Conclusion